In [1]:
# Author: bbaasan
# File: apr6_joining.ipynb
# Created: 2024-04-06
# Email: bbaasan@gmu.edu
# Purpose: find missing info of tickers not in EDGAR, NYSE, NASDAQ, OTC markets

In [44]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [3]:
aggregated = pd.read_pickle('D:/Earning2/data/aggregated.pkl')

In [4]:
aggregated.head()

,id,url,file_from,tagged,transcript,clean,date,stock,tick,meeting_date,...,sma200_close,sma50_vol,sma200_vol,target,delisted,acquisitions,bancrupt,changed,spinoff,ect
113946,1546,https://seekingalpha.com/article/930171-dover-...,0175,Dover Corporation (NYSE:DOV) Q3 2012 Earnings ...,[Dover Corporation (NYSE:DOV) Q3 2012 Earnings...,"[corporation, cerepak, financial, investor, an...",2012-10-17,NYSE,DOV,2012-10-17,...,32.404270,2394427.38,2340225.42,0,0,0,0,0,1,1
113952,1554,https://seekingalpha.com/article/929901-hallib...,0175,Halliburton (NYSE:HAL) Q3 2012 Earnings Call O...,[Halliburton (NYSE:HAL) Q3 2012 Earnings Call ...,"[halliburton, director, investor, financial, c...",2012-10-17,NYSE,HAL,2012-10-17,...,28.113018,12230288.00,14810852.00,1,0,0,0,0,0,1
113951,1553,https://seekingalpha.com/article/929911-csx-ma...,0175,CSX (NASDAQ:CSX) Q3 2012 Earnings Call October...,[CSX (NASDAQ:CSX) Q3 2012 Earnings Call Octobe...,"[investor, committee, marketing, operating, fi...",2012-10-17,NASDAQ,CSX,2012-10-17,...,6.178656,21269730.00,27394402.50,1,0,0,0,0,0,1
113940,1540,https://seekingalpha.com/article/930361-fulton...,0175,Fulton Financial (NASDAQ:FULT) Q3 2012 Earning...,[Fulton Financial (NASDAQ:FULT) Q3 2012 Earnin...,"[financial, fult, wakeley, committee, officio,...",2012-10-17,NASDAQ,FULT,2012-10-17,...,7.365196,1083936.00,1243970.50,1,0,0,0,0,0,1
117620,6876,https://seekingalpha.com/article/931171-nves-c...,0175,NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings C...,[NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings ...,"[nve, corporation, nvec, reynders, financial, ...",2012-10-17,NASDAQ,NVEC,2012-10-17,...,35.937213,15626.00,16996.00,1,0,0,0,0,0,1


#### NASDAQ

In [6]:
nas_listed = pd.read_csv('D:/Data/nasdaq/nasdaq/nasdaq_listed.csv')
nas_screener = pd.read_csv('D:/Data/nasdaq/nasdaq/nasdaq_screener.csv')
nas_traded = pd.read_csv('D:/Data/nasdaq/nasdaq/nasdaq_traded.csv')

In [7]:
from functools import reduce
df_nasdaq = [nas_traded, nas_listed, nas_screener]

In [8]:
nasdaq = reduce(lambda left, right: pd.merge(left, right, on='Symbol', how='left'), df_nasdaq)

#### NYSE

In [9]:
nyse_archive_listed = pd.read_csv('D:/Data/nyse_other_listings/archive/nyse_listed.csv')
nyse_archive_other = pd.read_csv('D:/Data/nyse_other_listings/archive/other_listed.csv')

In [10]:
nyse_archive = nyse_archive_listed.merge(nyse_archive_other, on='ACT_Symbol', how='right')

In [11]:
nyse_data_listed = pd.read_csv('D:/Data/nyse_other_listings/data/nyse_listed.csv')
nyse_data_other = pd.read_csv('D:/Data/nyse_other_listings/data/other-listed_csv.csv')

nyse_data_listed = nyse_data_listed.rename({'ACT_Symbol':'Symbol'},axis=1)
nyse_data_other = nyse_data_other.rename({'ACT Symbol':'Symbol'},axis=1)

In [12]:
nyse_data = nyse_data_other.merge(nyse_data_listed, on='Symbol', how='left')

#### Over-The-Counter Market

In [13]:
otc_screener = pd.read_csv('D:/Data/otc_stock_screener.csv')
otc_screener.head(2)

,Symbol,Security Name,Tier,Price,Change %,Vol,Sec Type,Country,State
0,AAAIF,ALTERNATIVE INVSTMENT TR,Pink Current,0.00,NaN,0,Fund,NaN,NaN
1,AAALF,AAREAL BANK AG AKT,Pink Current,31.99,NaN,0,Foreign Ordinary Shares,Germany,NaN


#### EDGAR

In [14]:
edgar = pd.read_json('D:/Data/company_tickers.json').T
edgar.head(2)

,cik_str,ticker,title
0,789019,MSFT,MICROSOFT CORP
1,320193,AAPL,Apple Inc.


In [15]:
aggregated = (aggregated
 .assign(nasdaq = lambda x: x['tick'].isin(nasdaq['Symbol'].values).astype(int))
 .assign(nyse = lambda x: x['tick'].isin(nyse_data['Symbol'].values).astype(int))
 .assign(otc = lambda x: x['tick'].isin(otc_screener['Symbol'].values).astype(int))
 .assign(edgar = lambda x: x['tick'].isin(edgar['ticker'].values).astype(int))
 )

In [19]:
aggregated.head()

,id,url,file_from,tagged,transcript,clean,date,stock,tick,meeting_date,...,delisted,acquisitions,bancrupt,changed,spinoff,ect,nasdaq,nyse,otc,edgar
113946,1546,https://seekingalpha.com/article/930171-dover-...,0175,Dover Corporation (NYSE:DOV) Q3 2012 Earnings ...,[Dover Corporation (NYSE:DOV) Q3 2012 Earnings...,"[corporation, cerepak, financial, investor, an...",2012-10-17,NYSE,DOV,2012-10-17,...,0,0,0,0,1,1,1,1,0,1
113952,1554,https://seekingalpha.com/article/929901-hallib...,0175,Halliburton (NYSE:HAL) Q3 2012 Earnings Call O...,[Halliburton (NYSE:HAL) Q3 2012 Earnings Call ...,"[halliburton, director, investor, financial, c...",2012-10-17,NYSE,HAL,2012-10-17,...,0,0,0,0,0,1,1,1,0,1
113951,1553,https://seekingalpha.com/article/929911-csx-ma...,0175,CSX (NASDAQ:CSX) Q3 2012 Earnings Call October...,[CSX (NASDAQ:CSX) Q3 2012 Earnings Call Octobe...,"[investor, committee, marketing, operating, fi...",2012-10-17,NASDAQ,CSX,2012-10-17,...,0,0,0,0,0,1,1,1,0,1
113940,1540,https://seekingalpha.com/article/930361-fulton...,0175,Fulton Financial (NASDAQ:FULT) Q3 2012 Earning...,[Fulton Financial (NASDAQ:FULT) Q3 2012 Earnin...,"[financial, fult, wakeley, committee, officio,...",2012-10-17,NASDAQ,FULT,2012-10-17,...,0,0,0,0,0,1,1,0,0,1
117620,6876,https://seekingalpha.com/article/931171-nves-c...,0175,NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings C...,[NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings ...,"[nve, corporation, nvec, reynders, financial, ...",2012-10-17,NASDAQ,NVEC,2012-10-17,...,0,0,0,0,0,1,1,0,0,1


In [21]:
for n, i in enumerate(aggregated.columns):
    print(n, i)

0 id
1 url
2 file_from
3 tagged
4 transcript
5 clean
6 date
7 stock
8 tick
9 meeting_date
10 close
11 volume
12 sma50_close
13 sma200_close
14 sma50_vol
15 sma200_vol
16 target
17 delisted
18 acquisitions
19 bancrupt
20 changed
21 spinoff
22 ect
23 nasdaq
24 nyse
25 otc
26 edgar


In [25]:
# extract cols need for analysis
working = (aggregated
.iloc[:, [3,6,7,8,17,18,19,20,21,22, 23, 24, 25, 26] ]
 )

In [26]:
(working
 .query(" delisted == 0 and acquisitions == 0 and bancrupt == 0 and changed == 0 and spinoff == 0 and ect == 0 and nasdaq == 0 and nyse == 0 and otc == 0 and edgar ==0") 
 )

,tagged,date,stock,tick,delisted,acquisitions,bancrupt,changed,spinoff,ect,nasdaq,nyse,otc,edgar
134639,Clean Coal Technologies Inc. (OTCQB:CCTC) Shar...,2013-03-27,OTCQB,CCTC,0,0,0,0,0,0,0,0,0,0
68827,Vedanta Resources Plc (OTCPK:VDNRF) F1Q14 Prod...,2013-07-31,OTCPK,VDNRF,0,0,0,0,0,0,0,0,0,0
19647,TETRAGON FINANCIAL (OTCPK:TGONF) 2013 Investor...,2013-08-11,OTCPK,TGONF,0,0,0,0,0,0,0,0,0,0
27444,Billabong International Ltd (OTCPK:BLLAF) F2Q2...,2014-02-20,OTCPK,BLLAF,0,0,0,0,0,0,0,0,0,0
75710,"ERF Wireless, Inc. (OTC:ERFB) Shareholder Conf...",2014-08-28,OTC,ERFB,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50918,Stolt-Nielsen Limited (OTCPK:SOIEF) Q2 2022 Ea...,2022-06-30,OTCPK,SOIEF,0,0,0,0,0,0,0,0,0,0
67713,Copper Mountain Mining Corporation. (CPPMF) Q2...,2022-07-25,noInfo,CPPMF,0,0,0,0,0,0,0,0,0,0
12476,"HTG Molecular Diagnostics, Inc. (NASDAQ:HTGM) ...",2022-08-11,NASDAQ,HTGM,0,0,0,0,0,0,0,0,0,0
106033,"Cengage Learning Holdings II, Inc. (OTC:CNGO) ...",2022-08-13,OTC,CNGO,0,0,0,0,0,0,0,0,0,0


In [27]:
print([(i, c) for i, c in enumerate(working.columns)])

[(0, 'tagged'), (1, 'date'), (2, 'stock'), (3, 'tick'), (4, 'delisted'), (5, 'acquisitions'), (6, 'bancrupt'), (7, 'changed'), (8, 'spinoff'), (9, 'ect'), (10, 'nasdaq'), (11, 'nyse'), (12, 'otc'), (13, 'edgar')]


In [28]:
missing = working[~working[['delisted', 'acquisitions', 'bancrupt', 'changed', 'spinoff', 'nasdaq', 'nyse', 'otc', 'edgar']].any(axis=1)]['tick'].value_counts().reset_index()
missing['above10'] = missing['count'].apply(lambda x: 1 if x >=10 else 0)

In [29]:
# filter ticks have less 10 earnings call transcripts
drop_ticks = (missing
.query("above10 == 0")
)
drop_ticks_list = drop_ticks['tick'].to_list()

In [ ]:
# filter ticks need more info. They have trasncripts more than 10
more_info_need_ticks = missing[missing['above10'] == 1]

In [ ]:
# filter those from the working dataset
working_count = working[working['tick'].isin(more_info_need_ticks['tick'].values)][['tagged','stock', 'tick']].groupby('tick').agg('count').reset_index()#.first().reset_index()#.to_csv('D:/Data/more_info_need_ticks.csv', index=False)
# create ticks first occurrance and number of records
working_first = working[working['tick'].isin(more_info_need_ticks['tick'].values)][['tagged','stock', 'tick']].groupby('tick').first().reset_index()
working_df = working_first.merge(working_count, on='tick', how='left')

In [31]:
#
aggregated2 = (aggregated
 .query("tick not in @drop_ticks_list ") # 329 rows will be dropped
 )
aggregated2.shape[0] - aggregated.shape[0]

-329

In [33]:
more_info_need_ticks = pd.read_csv("D:/Data/more_info_need_ticks.csv")

In [45]:
# create quarter period useing datetime column
quarters = pd.to_datetime(aggregated2.loc[:,'date']).dt.to_period('Q').astype(str)
# attach to the dataset
aggregated2.loc[:,'Q'] = quarters

In [47]:
aggregated2 = aggregated2.drop('ect', axis=1) # drop ect clumn since not know the source from

In [48]:
aggregated2

,id,url,file_from,tagged,transcript,clean,date,stock,tick,meeting_date,...,delisted,acquisitions,bancrupt,changed,spinoff,nasdaq,nyse,otc,edgar,Q
113946,1546,https://seekingalpha.com/article/930171-dover-...,0175,Dover Corporation (NYSE:DOV) Q3 2012 Earnings ...,[Dover Corporation (NYSE:DOV) Q3 2012 Earnings...,"[corporation, cerepak, financial, investor, an...",2012-10-17,NYSE,DOV,2012-10-17,...,0,0,0,0,1,1,1,0,1,2012Q4
113952,1554,https://seekingalpha.com/article/929901-hallib...,0175,Halliburton (NYSE:HAL) Q3 2012 Earnings Call O...,[Halliburton (NYSE:HAL) Q3 2012 Earnings Call ...,"[halliburton, director, investor, financial, c...",2012-10-17,NYSE,HAL,2012-10-17,...,0,0,0,0,0,1,1,0,1,2012Q4
113951,1553,https://seekingalpha.com/article/929911-csx-ma...,0175,CSX (NASDAQ:CSX) Q3 2012 Earnings Call October...,[CSX (NASDAQ:CSX) Q3 2012 Earnings Call Octobe...,"[investor, committee, marketing, operating, fi...",2012-10-17,NASDAQ,CSX,2012-10-17,...,0,0,0,0,0,1,1,0,1,2012Q4
113940,1540,https://seekingalpha.com/article/930361-fulton...,0175,Fulton Financial (NASDAQ:FULT) Q3 2012 Earning...,[Fulton Financial (NASDAQ:FULT) Q3 2012 Earnin...,"[financial, fult, wakeley, committee, officio,...",2012-10-17,NASDAQ,FULT,2012-10-17,...,0,0,0,0,0,1,0,0,1,2012Q4
117620,6876,https://seekingalpha.com/article/931171-nves-c...,0175,NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings C...,[NVE Corporation (NASDAQ:NVEC) F2Q13 Earnings ...,"[nve, corporation, nvec, reynders, financial, ...",2012-10-17,NASDAQ,NVEC,2012-10-17,...,0,0,0,0,0,1,0,0,1,2012Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,2954,https://seekingalpha.com/article/4610165-smart...,0015,Smartsheet Inc. (NYSE:SMAR) Q1 2024 Earnings C...,[Smartsheet Inc. (NYSE:SMAR) Q1 2024 Earnings ...,"[smartsheet, investor, godbole, financial, tru...",2023-06-08,NYSE,SMAR,2023-06-07,...,0,0,0,0,0,1,0,0,1,2023Q2
5565,2950,https://seekingalpha.com/article/4610226-mongo...,0015,"MongoDB, Inc. (NASDAQ:MDB) Annual William Blai...","[MongoDB, Inc. (NASDAQ:MDB) Annual William Bla...","[mongodb, research, analyst, tanjga, svp, mong...",2023-06-09,NASDAQ,MDB,2023-06-08,...,0,0,0,0,0,1,0,0,1,2023Q2
5566,2951,https://seekingalpha.com/article/4610193-tuya-...,0015,"Tuya, Inc. (NYSE:TUYA) Q1 2023 Earnings Confer...","[Tuya, Inc. (NYSE:TUYA) Q1 2023 Earnings Confe...","[director, svp, director, investor, director, ...",2023-06-09,NYSE,TUYA,2023-06-07,...,0,0,0,0,0,1,0,0,1,2023Q2
5567,2952,https://seekingalpha.com/article/4610188-trans...,0015,Transcontinental Inc. (OTCPK:TCLAF) wQ2 2023 E...,[Transcontinental Inc. (OTCPK:TCLAF) wQ2 2023 ...,"[otcpk, tclaf, director, marcoux, lecavalier, ...",2023-06-09,OTCPK,TCLAF,2023-06-08,...,0,0,0,0,0,0,0,1,0,2023Q2


In [49]:
more_info_need_ticks

,tick,tagged,stock,Country,Industry,isOperating,ddate,Source,Others,Unnamed: 9
0,AWLCF,Awilco Drilling PLC (OTC:AWLCF) Q1 2016 Earnin...,OTC,UK,Oil & Gas Drilling,Delisted,12/15/2022,https://finance.yahoo.com/quote/AWLCF/profile,https://naxs.se/en/awilco-drilling-plc-change-...,NaN
1,BPMUF,Basilea Pharmaceutica AG (OTC:BPMUF) H1 2017 E...,OTC,CHE,Biotechnology,Delayed,NaN,https://finance.yahoo.com/quote/BPMUF/,NaN,NaN
2,BSYBF,British Sky Broadcasting Group (OTCPK:BSYBF) Q...,OTCPK,UK,Communications,Delisted,11/7/2018,https://en.wikipedia.org/wiki/Sky_Group,NaN,NaN
3,CCIH,ChinaCache International Holdings Ltd. (CCIH) ...,noInfo,CHN,Internet Content,Delisted,9/10/2019,https://www.globenewswire.com/news-release/201...,NaN,NaN
4,CNGO,Cengage Learning Holdings II Inc. (OTC:CNGO) Q...,OTC,USA,NaN,NaN,NaN,NaN,NaN,NaN
5,CPPMF,Copper Mountain Mining Corporation. (CPPMF) Q1...,noInfo,CAN,NaN,Acquired,6/20/2023,link,NaN,NaN
6,CSUWF,"Enercare, Inc. (OTCPK:CSUWF) Q3 2014 Earnings ...",OTCPK,CAN,NaN,Acquired,10/16/2018,link,NaN,NaN
7,EFLVF,Electrovaya Inc. (EFLVF) Q2 2015 Earnings Conf...,noInfo,CAN,NaN,OTCQB: EFLVD or ELVA,7/6/2023,https://electrovaya.com/press/electrovaya-anno...,NaN,NaN
8,EORBF,Orbite Aluminae Inc. (OTC:EORBF) Q2 2014 Earni...,OTC,CAN,NaN,Acquired,4/6/2020,https://www.marketscreener.com/quote/stock/ORB...,NaN,NaN
9,FRRVF,Ferrovial SA (OTCPK:FRRVF) Q1 2017 Earnings Co...,OTCPK,NED,Transportation,NaN,NaN,NaN,NaN,NaN


In [65]:
aggregated2.loc[aggregated2['tick'] == 'AWLCF', 'delisted'] = 1
ex1 = aggregated2.query("tick == 'AWLCF' ").iloc[:, 6:]

In [63]:
pd.set_option('display.max_columns', None)

In [68]:
import plotly.express as px
fig = px.line(ex1, x='Q', y=['sma50_close', 'sma200_close'])
fig.show()

In [74]:
aggregated2.tick.value_counts().reset_index().to_csv('D:/Data/edgar_financials_needed.csv', index='False')